## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

C:\Users\Katli\.conda\envs\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Katli\.conda\envs\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Katli\.conda\envs\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Katli\.conda\envs\venv\lib\site-packages\tensorflow\python\framework\dtypes.

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(['EIN', 'NAME'], axis=1)
df.head()   

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
df.astype('object').describe(include='all').loc['unique', :]

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
Name: unique, dtype: object

In [5]:
# Look at APPLICATION_TYPE value counts for binning
df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Cutoff applications types with less than 500 entries 

app_counts = df['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
app_counts2 = app_counts.loc[app_counts['APPLICATION_TYPE'] < 500]    
application_types_to_replace = app_counts2['index'].tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1900        1
C4500        1
C6100        1
C2500        1
C1245        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts >1
df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C2400        6
C1720        6
C4100        6
C1600        5
C1257        5
C2710        3
C0           3
C1260        3
C1267        2
C1234        2
C1246        2
C1256        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Cutoff classifications with less than 1000 entries 

class_counts = df['CLASSIFICATION'].value_counts().to_frame().reset_index()
class_counts2 = class_counts.loc[class_counts['CLASSIFICATION'] < 1000]    
classifications_to_replace = class_counts2['index'].tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(df, prefix=['APPLICATION_TYPE', 'AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS'])
dummies

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [23]:
# Remove outliers from ASK_AMT column

# Explore range of values in ASK_AMT column
max_ask = dummies['ASK_AMT'].max()
min_ask = dummies['ASK_AMT'].min()
mean_ask = dummies['ASK_AMT'].median()
print(f'Min: {min_ask}')
print(f'Max: {max_ask}')
print(f'Median: {mean_ask}')

# Look for outliers in ASK_AMT column
Q1 = dummies['ASK_AMT'].quantile(0.25)
Q3 = dummies['ASK_AMT'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1-(1.5*IQR)
upper_bound = Q3+(1.5*IQR)
print(f'IQR:{IQR}')
print(f'Upper Bound: {upper_bound}')
print(f'Lower Bound: {lower_bound}')

# Remove outliers from dataset
dummies2 = dummies.loc[dummies['ASK_AMT'] < upper_bound]
dummies2.astype('object').describe(include='all').loc['unique', :]

# Over 8,000 outliers removed from ASK_AMT dataset

Min: 5000
Max: 8597806340
Median: 5000.0
IQR:2742.0
Upper Bound: 11855.0
Lower Bound: 887.0


STATUS                            2
ASK_AMT                         656
IS_SUCCESSFUL                     2
APPLICATION_TYPE_Other            2
APPLICATION_TYPE_T10              2
APPLICATION_TYPE_T19              2
APPLICATION_TYPE_T3               2
APPLICATION_TYPE_T4               2
APPLICATION_TYPE_T5               2
APPLICATION_TYPE_T6               2
APPLICATION_TYPE_T7               2
APPLICATION_TYPE_T8               2
AFFILIATION_CompanySponsored      2
AFFILIATION_Family/Parent         2
AFFILIATION_Independent           2
AFFILIATION_National              2
AFFILIATION_Other                 2
AFFILIATION_Regional              2
CLASSIFICATION_C1000              2
CLASSIFICATION_C1200              2
CLASSIFICATION_C2000              2
CLASSIFICATION_C2100              2
CLASSIFICATION_C3000              2
CLASSIFICATION_Other              2
USE_CASE_CommunityServ            2
USE_CASE_Heathcare                2
USE_CASE_Other                    2
USE_CASE_Preservation       

### Optimization Trial 1: Model dataframe with ASK_AMT outliers removed

In [24]:
# Split our preprocessed data into our features and target arrays
X = dummies2.drop(['IS_SUCCESSFUL'], axis=1)
y = dummies2['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [26]:
# Create the Keras Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
19569/19569 [==============================] - 1s 51us/sample - loss: 0.5558 - acc: 0.7343
Epoch 2/100
19569/19569 [==============================] - 1s 44us/sample - loss: 0.5394 - acc: 0.7446
Epoch 3/100
19569/19569 [==============================] - 1s 47us/sample - loss: 0.5366 - acc: 0.74300s - loss: 0.5400 - 
Epoch 4/100
19569/19569 [==============================] - 1s 47us/sample - loss: 0.5337 - acc: 0.7439
Epoch 5/100
19569/19569 [==============================] - 1s 45us/sample - loss: 0.5339 - acc: 0.7460
Epoch 6/100
19569/19569 [==============================] - 1s 45us/sample - loss: 0.5326 - acc: 0.7443
Epoch 7/100
19569/19569 [==============================] - 1s 47us/sample - loss: 0.5319 - acc: 0.74600s - loss: 0.5291 - acc: 
Epoch 8/100
19569/19569 [==============================] - 1s 48us/sample - loss: 0.5309 - acc: 0.7468
Epoch 9/100
19569/19569 [==============================] - 1s 44us/sample - loss: 0.5307 - acc: 0.7472
Epoch 10/100
19569/19569 [==

19569/19569 [==============================] - 1s 47us/sample - loss: 0.5205 - acc: 0.7530
Epoch 77/100
19569/19569 [==============================] - 1s 44us/sample - loss: 0.5208 - acc: 0.7521
Epoch 78/100
19569/19569 [==============================] - 1s 48us/sample - loss: 0.5204 - acc: 0.7529
Epoch 79/100
19569/19569 [==============================] - 1s 49us/sample - loss: 0.5204 - acc: 0.7523
Epoch 80/100
19569/19569 [==============================] - 1s 49us/sample - loss: 0.5202 - acc: 0.7526
Epoch 81/100
19569/19569 [==============================] - 1s 49us/sample - loss: 0.5207 - acc: 0.7529
Epoch 82/100
19569/19569 [==============================] - 1s 52us/sample - loss: 0.5202 - acc: 0.7525
Epoch 83/100
19569/19569 [==============================] - 1s 46us/sample - loss: 0.5202 - acc: 0.75270s - loss: 0.5215 - - ETA: 0s - loss: 0.5186 - acc: 0.
Epoch 84/100
19569/19569 [==============================] - 1s 48us/sample - loss: 0.5202 - acc: 0.7528
Epoch 85/100
19569/1956

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6524/6524 - 0s - loss: 0.5357 - acc: 0.7520
Loss: 0.53574636013942, Accuracy: 0.7519926428794861


In [63]:
# Export our model to HDF5 file
nn.save_weights('AlphabetSoupCharity_Optimization.hdf5')

### Optimization Trial 2: Change model parameters, with outliers removed

In [53]:
# Number of nuerons in hidden layers changed in this model

# Split our preprocessed data into our features and target arrays
X = dummies2.drop(['IS_SUCCESSFUL'], axis=1)
y = dummies2['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create the Keras Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 50)                2200      
_________________________________________________________________
dense_16 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 11        
Total params: 2,721
Trainable params: 2,721
Non-trainable params: 0
_________________________________________________________________


In [54]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [61]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
19569/19569 [==============================] - 1s 73us/sample - loss: 0.5158 - acc: 0.7543
Epoch 2/100
19569/19569 [==============================] - 1s 73us/sample - loss: 0.5158 - acc: 0.7546
Epoch 3/100
19569/19569 [==============================] - 1s 68us/sample - loss: 0.5158 - acc: 0.7549
Epoch 4/100
19569/19569 [==============================] - 1s 68us/sample - loss: 0.5159 - acc: 0.7545
Epoch 5/100
19569/19569 [==============================] - 1s 60us/sample - loss: 0.5157 - acc: 0.7542
Epoch 6/100
19569/19569 [==============================] - 1s 45us/sample - loss: 0.5163 - acc: 0.7539
Epoch 7/100
19569/19569 [==============================] - 1s 42us/sample - loss: 0.5158 - acc: 0.7545
Epoch 8/100
19569/19569 [==============================] - 1s 45us/sample - loss: 0.5157 - acc: 0.7551
Epoch 9/100
19569/19569 [==============================] - 1s 43us/sample - loss: 0.5158 - acc: 0.7546
Epoch 10/100
19569/19569 [==============================] - 1s 43us/sampl

19569/19569 [==============================] - 1s 43us/sample - loss: 0.5156 - acc: 0.75470s - loss: 0.5114 - ac
Epoch 78/100
19569/19569 [==============================] - 1s 49us/sample - loss: 0.5155 - acc: 0.7550
Epoch 79/100
19569/19569 [==============================] - 1s 55us/sample - loss: 0.5158 - acc: 0.7544
Epoch 80/100
19569/19569 [==============================] - 1s 52us/sample - loss: 0.5157 - acc: 0.7543
Epoch 81/100
19569/19569 [==============================] - 1s 46us/sample - loss: 0.5168 - acc: 0.7544
Epoch 82/100
19569/19569 [==============================] - 1s 45us/sample - loss: 0.5154 - acc: 0.7550
Epoch 83/100
19569/19569 [==============================] - 1s 45us/sample - loss: 0.5155 - acc: 0.7543
Epoch 84/100
19569/19569 [==============================] - 1s 47us/sample - loss: 0.5154 - acc: 0.7546
Epoch 85/100
19569/19569 [==============================] - 1s 42us/sample - loss: 0.5155 - acc: 0.7547
Epoch 86/100
19569/19569 [=============================

In [62]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6524/6524 - 0s - loss: 0.5420 - acc: 0.7498
Loss: 0.5420252736143946, Accuracy: 0.7498466968536377


### Optimization Trial 3: Bin ASK_AMT data, keeping outliers 

In [36]:
# Create bins for ASK_AMT column
bins = [5000,10000,15000,20000,25000,30000,35000,40000,100000,1000000]
group_name = ['5,000-10000','>10,000-15,000','>15,000-20,000','>20,000-25,000','>25,000-30,000','>30,000-35,000','>35,000-40,000','>40,000-100,000','>100,000-1,000,000']
dummies['ASK_Bins']=pd.cut(dummies['ASK_AMT'],bins,labels=group_name,include_lowest=True)

# Drop ASK_AMT column
dummies3 = dummies.drop(['ASK_AMT'], axis=1)

# Convert bin column to numeric values 
dummies3 = pd.get_dummies(dummies3, prefix=['ASK_Bins'])
dummies3.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,SPECIAL_CONSIDERATIONS_Y,"ASK_Bins_5,000-10000","ASK_Bins_>10,000-15,000","ASK_Bins_>15,000-20,000","ASK_Bins_>20,000-25,000","ASK_Bins_>25,000-30,000","ASK_Bins_>30,000-35,000","ASK_Bins_>35,000-40,000","ASK_Bins_>40,000-100,000","ASK_Bins_>100,000-1,000,000"
0,1,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [37]:
# Split our preprocessed data into our features and target arrays
X = dummies3.drop(['IS_SUCCESSFUL'], axis=1)
y = dummies3['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)

In [38]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [47]:
# Create the Keras Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=51))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 80)                4160      
_________________________________________________________________
dense_10 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 31        
Total params: 6,621
Trainable params: 6,621
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [49]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5704 - acc: 0.7212
Epoch 2/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5523 - acc: 0.7309
Epoch 3/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5486 - acc: 0.7320
Epoch 4/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5461 - acc: 0.7332
Epoch 5/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5443 - acc: 0.73300s - loss: 0.5455 - acc:
Epoch 6/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5424 - acc: 0.7352
Epoch 7/100
25724/25724 [==============================] - 2s 66us/sample - loss: 0.5420 - acc: 0.7363
Epoch 8/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5411 - acc: 0.7357
Epoch 9/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5404 - acc: 0.7361
Epoch 10/100
25724/25724 [=======================

25724/25724 [==============================] - 1s 51us/sample - loss: 0.5227 - acc: 0.7456
Epoch 75/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5221 - acc: 0.7448
Epoch 76/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5230 - acc: 0.7446
Epoch 77/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5227 - acc: 0.7455
Epoch 78/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5225 - acc: 0.7436
Epoch 79/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5218 - acc: 0.74440s - loss: 0.5215 - acc: 
Epoch 80/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5225 - acc: 0.74441s - los
Epoch 81/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5211 - acc: 0.7445
Epoch 82/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5217 - acc: 0.74530s - loss: 0.5082 
Epoch 83/100
25724/25724 [

In [60]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6524/6524 - 0s - loss: 0.5416 - acc: 0.7503
Loss: 0.5416244672603362, Accuracy: 0.7503065466880798
